# TP2 Prévision de consommation avec réseau de neurones


<img src="pictures/Présentation_FormationIA_TPDeepLearning.png" width=1000 >

**Dans l'épisode précédent**  

Nos modèles de régression du TP1 nous ont donné des premiers résultats et des premières intuitions sur notre problème de prévision de consommation pour le lendemain. 

Nous avons pu analyser des profils de courbe de consommation au jour, à la semaine, au mois. Nous avons également observé la dépendance entre la consommation et la consommation retardée. Nous avons aussi vu l'impact des jours fériés. 

Nous avons ensuite utilisé de premiers modèles en machine learning pour apprendre par observations l'influence de différents contextes sur la consommation sans les décrires explicitement selon des lois. Nous sommes arrivés à une erreur moyenne de test de 2,8%, bien mieux que ce qui avait été obtenu via une approche naïve.

Des difficultés se sont posées pour intégrer les variables météorologiques très dépendantes entre elles et pour intégrer un vecteur de consommation retardée.

Avec l'approche classique exposée dans ce TP1, nous avons en particulier constaté le besoin d'une expertise et d'un travail autour des variables explicatives pour obtenir un modèle performant.

**Aujourd'hui** 

Nous allons de nouveau nous attaquer à ce sujet de la prévision de consommation nationale pour le lendemain, mais cette fois en utilisant un modèle de prévision par réseau de neurones. Nous allons exploiter leur capacité à capter ces phénomènes non-linéaires et interdépendants. Nous allons mettre en évidence le moindre besoin en feature engineering en travaillant directement à la granularité de la donnée, sans créer de variables agrégées ou transformées par de l'expertise.

**Ce que vous allez voir dans ce second TP**

- Un rappel de notre problème et récapitulatif des performances de nos modèles précédents
- Une nouvelle méthode numérique pour préparer ses données et faciliter l'apprentissage : la normalisation
- La création d'un premier réseau de neurones pour prédire la consommation dans 24h
- L'utilisation de tensorboard pour observer en temps réel la courbe d'apprentissage du réseau de neurones
- La création de modèles de plus en plus performants en intégrant davantage d'informations dans notre modélisation
- L'évaluation des modèles sur 2 types de jeux de test

**Ce que vous allez devoir faire**

- Compléter les quelques trous de codes que nous vous avons laissé si vous le souhaitez. La solution est disponible dans le TP complété.
- Répondre aux quelques questions disséminées dans ce TP
- Entrainer votre propre modèle pour améliorer les performances d'un modèle existant et essayer de remporter notre mini-challenge !

__NB__ : Pour ce TP nous utiliserons Keras, une bibliothèque python de haut niveau qui appelle des fonctions de la librairie TensorFlow. D'autres librairies existent, Keras a été retenue en raison de sa facilité d'utilisation.

# Dimensionnement en temps
La durée estimée de ce TP est d'environ 1h30 :
- 10 minutes pour charger les données pour les réseaux de neurones 
- 20 minutes pour entrainer un premier modèle de réseau de neurones, en examiner le code implémentant ce réseau de neurones
- Le reste pour jouer et tenter d'améliorer la qualité de la prédiction avec de nouvelles variables explicatives, ou en choisissant d'autres hyper-paramètres. 

# I) Préparation des données

## Chargement des librairies nécessaires

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime
import zipfile
import requests, io #get data from url
from urllib.request import urlopen
import joblib
import tempfile

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, iplot_mpl
import matplotlib.pyplot as plt

# sklearn est la librairie de machine learning en python et scipy une librairie statistiques
from sklearn.metrics import mean_squared_error

# Keras est la librairie que nous utilisons pour se créer des modèles de réseau de neurones
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Activation, Embedding, LSTM, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import tensorflow.keras.utils as tf_utils
K.set_image_data_format('channels_last')

%matplotlib inline

init_notebook_mode(connected=True)

import pydot
import graphviz
from IPython.display import display_png

# set seed for rng
tf_utils.set_random_seed(
    42
)

: 

In [ ]:
isInColab = True #False if in Jupyter

if(isInColab):#to have iplot working in colab
    import plotly.io as pio
    #pio.renderers
    pio.renderers.default = 'colab'

## I) Récupération et préparation des données

Dans cette partie nous allons charger les fichiers csv nécessaires pour l'analyse, puis les convertir en data-frame python. Les données de base à récupérer sont :
- la base de données issues du TP1 (Les historiques de consommation, leur lag, les données météo en température, leur lag, les jours feriés) 

En terme de transformation des données pour mieux les préparer:

- nous allons aussi voir comment normaliser les données, une transformation souvent bien utile en pratique pour une meilleure convergence numérique. 

Cela vient compléter les transformations vu précédemment pour les données calendaires, et aussi la transformation "one-hot" pour les données catégorielles

In [ ]:
isDataFromGithub = False #True especially if using colab as it only download the notebook and not the entire github repository. Otherwise with Binder or in local, set it False
if isInColab:
    isDataFromGithub = True

if(isDataFromGithub):
    data_folder = 'https://raw.githubusercontent.com/rte-france/Formation_FIFA/master/data'#data depuis github
else:
    data_folder = os.path.join(os.getcwd(), "data") #data en local


### Récupération de nos variables à prédire: la consommation française

In [ ]:
y_csv = os.path.join(data_folder, "y_conso_tp2.csv")
y = pd.read_csv(y_csv, sep=";", engine='c', header=0)

y['ds'] = pd.to_datetime(y['ds'], utc=True)

display(y.head(5))
print(y.shape)

<font color='green'>

* Petit rappel, que fait la fonction "shape" ?

</font>

Les données météo sont confidentielles, et donc ont été cryptées. Pour les lire vous avez besoin d'un mot de passe qui ne peut vous être donné que dans le cadre d'un travail au sein de RTE.

In [ ]:
password = ""

In [ ]:
# Récupération des températures et jours fériés
x_zip = os.path.join(data_folder, "x_input_tp2.zip")

if(isDataFromGithub):
    ####data sur github
    r = requests.get(x_zip)
    x_zip_object = zipfile.ZipFile(io.BytesIO(r.content))
else:
    ######data en local
    x_zip_object = zipfile.ZipFile(x_zip)
    
x_zip_object.setpassword(bytes(password,'utf-8'))
x = pd.read_csv(x_zip_object.open('x.csv'), sep=";", engine='c', header=0)

x['ds'] = pd.to_datetime(x['ds'], utc=True)

display(x.head(5))
print(x.shape)

In [ ]:
# On récupère aussi le scaler qui permet de dénormaliser la prédiction du réseau de neurones
if(isDataFromGithub):
    scaler_conso_nat = joblib.load(urlopen(os.path.join(data_folder, "scaler_conso.save")))
else:
    scaler_conso_nat = joblib.load(os.path.join(data_folder, "scaler_conso.save"))

<font color='blue'>

**A propos de la normalisation...**


</font>

En théorie, la normalisation des données d'entrée n'est pas indispensable pour entrainer un réseau de neurones.  

En effet, on devrait apprendre des poids et biais plus ou moins importants pour équilibrer les contributions des différentes variables explicatives en entrée. 

Cependant en pratique, normaliser les données d'entrée permet généralement d'obtenir un apprentissage plus rapide du réseau de neurones.

<br/>
<font color='green'>
    
* Comment l'expliquez-vous ?

<font>

## II) Création des jeux d'apprentissage, de validation, et de test

<font color='green'>
    
**Question** : 
* A quoi servent les jeux d'entrainement, de validation, et de test ?

</font>

Nous allons nous créer les jeux de données suivants :
* Jeu d'entrainement : 90% des points pris aléatoirement entre le début du dataset et le 31 décembre 2017
* Jeu de validation : les 10% restant des points entre le début du dataset et le 31 décembre 2017
* Jeu de test : tous les points horaires à partir du 1er janvier 2018

In [ ]:
# D'abord on repère les lignes de chacun des set

TEST_START_DATE = datetime.datetime(year=2018, month=1, day=1, tzinfo=datetime.timezone.utc)

mask_test_set = (x["ds"] >= TEST_START_DATE)

mask_train_validation = (x['ds'] < TEST_START_DATE)
mask_train_validation = mask_train_validation.astype(bool)

def filter(value, threshold):
    return True if value < threshold else False
    
mask_train_set = [filter(value, 0.9) for value in np.random.uniform(0, 1, size=x.shape[0])] & mask_train_validation
mask_validation_set = ~mask_train_set & mask_train_validation

# petite verif
print(x.shape)
print("Nombre d'éléments dans le train set : " + str(np.sum(mask_train_set)))
print("Nombre d'éléments dans le validation set : " + str(np.sum(mask_validation_set)))
print("Nombre d'éléments dans le test set : " + str(np.sum(mask_test_set)))
print(np.sum(mask_train_set) + np.sum(mask_validation_set) + np.sum(mask_test_set))

In [ ]:
print(x.columns)
x.head(2)

In [ ]:
# Puis on constitue les sets
x_train_full = x[mask_train_set]
x_validation_full = x[mask_validation_set]
x_test_full = x[mask_test_set]

y_train_full = y[mask_train_set]
y_validation_full = y[mask_validation_set]
y_test_full = y[mask_test_set]

x_train_full.reset_index(inplace=True, drop=True)
x_validation_full.reset_index(inplace=True, drop=True)
x_test_full.reset_index(inplace=True, drop=True)
y_train_full.reset_index(inplace=True, drop=True)
y_validation_full.reset_index(inplace=True, drop=True)
y_test_full.reset_index(inplace=True, drop=True)

In [ ]:
print("Shape de x_train_full : " + str(x_train_full.shape))
print("Shape de x_validation_full : " + str(x_validation_full.shape))
print("Shape de x_test_full : " + str(x_test_full.shape))

print("Shape de y_train : " + str(y_train_full.shape))
print("Shape de y_validation : " + str(y_validation_full.shape))
print("Shape de y_test : " + str(y_test_full.shape))

In [ ]:
y_light_columns = ["conso_real_scaled"]

y_train_light = y_train_full[y_light_columns]
y_validation_light = y_validation_full[y_light_columns]
y_test_light = y_test_full[y_light_columns]

# III) Getting started with Keras API

Jusqu'ici, nous avons importé nos données. Nous les avons ensuite préparées pour les fournir au réseau de neurones (one-hot encoding, normalisation). Nous avons également créé nos jeux d'entrainement, validation, et de test.

Il est maintenant l'heure de se construire un réseau de neurones, de l'entrainer, et de lui faire faire des prédictions !

Dans cette partie III) nous allons nous familiariser avec la librairie Keras qui permet d'implémenter des réseaux de neurones, puis en partie IV) nous l'appliquerons à notre problématique de prévision de consommation.

**Cette partie III) est générique et indépendante de notre problématique de prévision de consommation**

<img src="pictures/FirstNeuralNetwork.jpeg" width=700 >

## Deux fonctions bien utiles

Nous allons commencer par implémenter deux fonctions que nous appellerons pour chacun des modèles que nous allons tester:
- Fonction 1: **new_keras_model**, pour instancier un modèle de réseau de neurone avant apprentissage
- Fonction 2: **plot_neural_net**, pour visualiser un réseau de neurones

### Création d'une architecture de réseau de neurones

In [ ]:
def new_keras_model(n_inputs, n_outputs=1, hidden_layers=None, activation='relu'):
    """      
    arguments
        - n_inputs : le nombre de features en entrée
        - n_outputs : le nombre de sorties (variables à prédire)
        - hidden_layers : une liste. 
                          La taille de la liste donne le nombre de couches cachées.
                          Les éléments de la liste donnent le nombre de neurones par couche.
                          Cette liste doit contenir au moins un élément
        - activation: `str` "relu" ou "sigmoid" le type de "non linéarité" / "fonction d'activation"
                      que vous voulez utiliser.
        
    returns
        - un objet de type Model 
    """
    model = Sequential()
    
    input_dim = n_inputs
    print(n_inputs)
    for l_size in hidden_layers:
        model.add(Dense(l_size, input_dim=input_dim, activation=activation))
        input_dim = l_size

    # Pour une régression, la fonction d'activation finale est simplement la fonction identité
    model.add(Dense(n_outputs, input_dim=input_dim, activation='linear'))  
    
    return model

### Inspection de l'architecture d'un reseau de neurones
On se créé un réseau avec un certains nombre de couches qui peuvent chacune avoir différentes dimensions. On peut ensuite inspecter les dimensions et le nombre de paramètres de ce réseau avec la méthode _summary_ de Keras. 

In [ ]:
# on se crée un réseau de neurones avec un certains nombre d'entrées et sorties
n_inputs = 8  #un choix raisonnable pour visualiser ce modèle ensuite
n_outputs = 1

hidden_layers = [10, n_inputs, 6]
dummy_model = new_keras_model(n_inputs, n_outputs, hidden_layers=hidden_layers)
dummy_model.summary()

Créons-nous maintenant une fonction pour dessiner ce réseau de neurones.

Ne vous embêtez pas trop à comprendre le code de la fonction *plot_neural_net*.

In [ ]:
def plot_neural_net(model):
    layers = [model.input_shape[1]]
    for layer in model.layers:
        layers.append(layer.get_output_at(0).get_shape().as_list()[1])
        
    tmp_file = os.path.join(tempfile.gettempdir(), 'out.dot')                
    with open(tmp_file, 'w') as f:       
        layers_str = ["Input"] + ["Hidden"] * (len(layers) - 2) + ["Output"]
        layers_col = ["none"] + ["none"] * (len(layers) - 2) + ["none"]
        layers_fill = ["black"] + ["gray"] * (len(layers) - 2) + ["black"]
        penwidth = 15
        font = "Hilda 10"
        print("digraph G {",file=f)
        print("\tfontname = \"{}\"".format(font),file=f)
        print("\trankdir=LR",file=f)
        print("\tsplines=line",file=f)
        print("\tnodesep=.08;",file=f)
        print("\tranksep=1;",file=f)
        print("\tedge [color=black, arrowsize=.5];",file=f)
        print("\tnode [fixedsize=true,label=\"\",style=filled," + \
              "color=none,fillcolor=gray,shape=circle]\n",file=f)

        # Clusters
        for i in range(0, len(layers)):
            print(("\tsubgraph cluster_{} {{".format(i)),file=f)
            print(("\t\tcolor={};".format(layers_col[i])),file=f)
            print(("\t\tnode [style=filled, color=white, penwidth={},"
                   "fillcolor={} shape=circle];".format(
                penwidth,
                layers_fill[i])),file=f)
            print(("\t\t"), end=' ',file=f)
            for a in range(layers[i]):
                print("l{}{} ".format(i + 1, a), end=' ',file=f)

            print(";",file=f)
            print(("\t\tlabel = {};".format(layers_str[i])),file=f)
            print("\t}\n",file=f)

        # Nodes
        for i in range(1, len(layers)):
            for a in range(layers[i - 1]):
                for b in range(layers[i]):
                    print("\tl{}{} -> l{}{}".format(i, a, i + 1, b), file=f)
        print("}", file=f)
    
    dot = graphviz.Source.from_file(tmp_file, engine='dot', format="png")
    return dot


Visualisons le reseau de neurone test créé précédemment :

In [ ]:
plot_neural_net(dummy_model)

ATTENTION: Pour des grandes tailles de reseau, cette visualisation n'est pas adaptée et le temps d'éxécution de cette fonction sera très long !

<font color='green'>
    
**Défi !** : 
* Créez vous un reseau de neurones en forme de noeud papillon, dont la couche de sortie fait la même dimension que la couche d'entrée.

</font>

In [ ]:
# votre new_keras_model à créer ici
model_noeud_papillon = None

In [ ]:
#plot_neural_net(model_noeud_papillon)

Bravo ! Vous venez de créer un réseau de neurone d'une classe très particulière: c'est un autoencoder !
Pour les curieux, vous pouvez retrouver le bestiaire des réseaux de neurones ici: https://towardsdatascience.com/the-mostly-complete-chart-of-neural-networks-explained-3fb6f2367464

## IV) Un premier modèle de réseau de neurones

## Choix des variables explicatives

pour ce TP, nous avons un jeu d'entrée X contenant beaucoup de variables. Afin de commencer par un modèle simple, nous allons élaguer ce X pour réduire le nombre de features en entrée. Dans ce TP, nous allons donc lister les colonnes à retirer des datasets X initialisés ci-dessus.

Pour un cas d'étude réel, une approche pragmatique serait de commencer par se créer un premier X simple, de voir les performances du modèle, puis ensuite d'incorporer de plus en plus de features dans le X pour évaluer la progression des performances de nos modèles.

Toutefois, en deep learning, il est courant commencer directement en mettant en entrée toute l'information disponible. En effet une des forces des réseaux de neurones est leur capacité à "digérer" la donnée, en se nourrissant d'informations redondantes.

pour des raisons pédagogiques, nous allons commencer avec la première approche.

Pour le premier réseau de neurones que nous allons entrainer, nous allons simplement garder les variables calendaires ainsi que la valeur de consommation nationale réalisée la veille.

In [ ]:
# Petit rappel pour se remettre en mémoire les variables que nous avons à disposition
x_train_full.head()

In [ ]:
# Sélectionnons un sous-ensemble de ces variables
x_light_columns = [elt for elt in x.columns if elt not in 
                   ["ds", "is_bank_holiday", "temperature_real_24h_avant", "temperature_prevue", "conso_real_24h_avant"]
                  ]

x_train_light = x_train_full[x_light_columns]
x_validation_light = x_validation_full[x_light_columns]
x_test_light = x_test_full[x_light_columns]

In [ ]:
print("Shape de x_train_light : " + str(x_train_light.shape))
print("Shape de x_validation_light : " + str(x_validation_light.shape))
print("Shape de x_test_light : " + str(x_test_light.shape))

print("Shape de y_train_light : " + str(y_train_light.shape))
print("Shape de y_validation_light : " + str(y_validation_light.shape))
print("Shape de y_test_light : " + str(y_test_light.shape))

## Création du réseau de neurones et hyper-paramétrage
Un réseau de neurones profond est constuitué d'un certains nombre de couches, chacune portant un certain nombre de neurones. Ce sont 2 hyperparamètres que vous pouvez faire varier et qui vous permettront d'obtenir un apprentissage plus ou moins précis, en utilisant plus ou moins de puissance de calcul.

Le "learning rate" de l'optimiseur est également un hyperparamètre qui influencera la convergence et la vitesse de convergence de l'apprentissage, où l'on cherche à optimiser notre modèle pour minimiser l'erreur de prédiction. 

In [ ]:
n_inputs = x_train_light.shape[1]  # nombre de features en entrée du réseau de neurones
n_outputs = y_train_light.shape[1]
hidden_layers = [n_inputs, n_inputs, n_inputs, n_inputs, n_inputs]

first_model = new_keras_model(n_inputs, n_outputs, hidden_layers)

In [ ]:
# on affiche le nombre de paramètres de ce modèle avec la fonction summary de Keras
first_model.summary()

In [ ]:
first_model.compile(
    loss='mean_squared_error', 
    optimizer=Adam(lr=0.001), 
)

In [ ]:
# On crée ici une instance de l'utilitaire tensorboard qui va nous permettre de visualiser 
# les courbes d'apprentissage de nos différents modèles.
# On reviendra avec plus d'explication sur TensorBoard un peu plus tard.
# Donner un nom a votre modele pour le retrouver dans les logs tensorboard
model_name = "my_first_model_" + datetime.datetime.now().strftime("%H-%M-%S")
tensorboard = TensorBoard(log_dir="logs/{}".format(model_name))
    

## Entrainement

La cellule suivante peut prendre un peu de temps à s'exécuter. On reconnait là la méthode **fit** commune à chaque modèle de machine-learning pour entraîner son modèle.

In [ ]:
# Paramètres d'appel
# - epoch: on précise le nombre d'epochs (le nombre de fois que l'on voit le jeu d'apprentissage en entier)
# - batch size: le nombre d'exemples sur lequel on fait un "pas" d'apprentissage parmi tout le jeu
# - validation_split: la proportion d'exemples que l'on conserve pour notre jeu de validation
# - callbacks: pour appeler des utilitaires/fonctions externes pour récupérer des résultats
first_model.fit(
    x_train_light, 
    y_train_light, 
    epochs=100, 
    batch_size=100, 
    validation_data=(x_validation_light, y_validation_light),
    callbacks=[tensorboard]
)

<font color='green'>

* D'après les informations de logs exposées ici, quelle semble être la perfomance atteinte par votre réseau de neurones ? 

</font>

# Tensorboard
c'est un utilitaire de tensorflow qui permet de visualiser en temps réel les courbes d'apprentissage des réseau de neurones et est donc utile pour arrêter l'apprentissage si les progrès sont faibles.

En particulier, vous pouvez vous intéresser à la courbe de l'erreur (loss) d'entrainement et de validation pour visualiser la progression de l'apprentissage et une tendance au surapprentissage en fin d'apprentissage.

<img src="pictures/CourbesTensorboard.png" width=1000 >

**Pour ouvrir une fenêtre tensorboard, revenez sur la page d'accueil de Jupyter, placez vous dans le dossier logs dans lequel se trouve les logs de vos entrainement, puis cliquez sur New (en haut à droite) et enfin sur Tensorboard**.

Une fenêtre pop-up doit s'ouvrir. Si elle est bloquée, autorisez son ouverture.

In [ ]:
#or you can directly load tensorboard in the notebook - especially if you are on Colab and not in Jupyter
%load_ext tensorboard
%tensorboard --logdir logs


<font color='green'>

* Vous devriez visualiser les courbes de 2 modèles: celui que vous venez d'entrainer et un modèle qui avait été entrainé de la même manière mais avec des données non normalisée. Que constatez-vous ? Comment l'expliquez-vous ?
<br/><br/>
* Il se passe quelque chose d'étonnant vers l'epoch 50. Qu'est-ce que cela vous inspire ?

</font>

## Evaluation de la qualité du modèle

In [ ]:
predictions_train_scaled = first_model.predict(x_train_light)
predictions_val_scaled = first_model.predict(x_validation_light)
predictions_test_scaled = first_model.predict(x_test_light)

predictions_train = scaler_conso_nat.inverse_transform(predictions_train_scaled).reshape(-1)
predictions_val = scaler_conso_nat.inverse_transform(predictions_val_scaled).reshape(-1)
predictions_test = scaler_conso_nat.inverse_transform(predictions_test_scaled).reshape(-1)

print(predictions_test)

In [ ]:
relative_error_on_train = np.abs((y_train_full['conso_real'] - predictions_train) / y_train_full['conso_real'])
mean_error_on_train = np.mean(relative_error_on_train)
max_error_on_train = np.max(relative_error_on_train)
rmse = np.sqrt(mean_squared_error(y_train_full['conso_real'], predictions_train))

print("Erreur moyenne sur le jeu de train : " + str(mean_error_on_train * 100) + " %")
print("Erreur max sur le jeu de train : " + str(max_error_on_train * 100) + " %")
print("RMSE : " + str(rmse) + " MW")

In [ ]:
relative_error_on_val = np.abs((y_validation_full['conso_real'] - predictions_val) / y_validation_full['conso_real'])
mean_error_on_val = np.mean(relative_error_on_val)
max_error_on_val = np.max(relative_error_on_val)
rmse = np.sqrt(mean_squared_error(y_validation_full['conso_real'], predictions_val))

print("Erreur moyenne sur le jeu de validation : " + str(mean_error_on_val * 100) + " %")
print("Erreur max sur le jeu de validation : " + str(max_error_on_val * 100) + " %")
print("RMSE : " + str(rmse) + " MW")

In [ ]:
relative_error_on_test = np.abs((y_test_full['conso_real'] - predictions_test) / y_test_full['conso_real'])
mean_error_on_test = np.mean(relative_error_on_test)
max_error_on_test = np.max(relative_error_on_test)
rmse = np.sqrt(mean_squared_error(y_test_full['conso_real'], predictions_test))

print("Erreur moyenne sur le jeu de test : " + str(mean_error_on_test * 100) + " %")
print("Erreur max sur le jeu de test : " + str(max_error_on_test * 100) + " %")
print("RMSE : " + str(rmse) + " MW")

In [ ]:
iplot([{"x": y_test_full['ds'], "y": y_test_full['conso_real'], "name": "realise"},
       {"x": y_test_full['ds'], "y": predictions_test, "name": "prevision"}
      ])

L'erreur est ici comparable à celle des autres modèles en machine Learning (random forest, xgboost). Cela peut nous conforter dans le fait que notre réseau de neurones s'est créé de bonnes représentations pour ces variables calendaires. 

La différence en performance peut devenir plus flagrante lorsque l'on intègre des variables à une maille très granulaire (les pixels d'une images, la température dans toutes les villes de France) avec une forte interdépendance.

Pour inspecter dynamiquement des visualisations, la librairie plotly se révèle très utile.
Ci-dessous vous pouvez identifier les jours et heures qui présentent les erreurs les plus importantes pour ensuite imaginer ce qui a pu pêcher.

In [ ]:
iplot([{"x": y_test_full['ds'], "y": relative_error_on_test}])

<font color='green'>

* Quelles sont les heures ou les journées avec les erreurs les plus importantes. Avez-vous une idée à quoi pourrait correspondre ces heures ou ces jours ?

</font>

# V) A vous de jouer, faites fonctionner vos neurones naturels

<img src="pictures/we-need-you.png" width=500 >

# **Challenge**: entrainez et testez votre nouveau modèle avec de nouvelles variables et paramètres choisies

N'hésitez pas à largement copier-coller des morceaux de code ci-dessus ;-)  
Venez partager vos investigations sur cette google sheet : https://docs.google.com/spreadsheets/d/1oIx8jjzIh7Ugp3ZJMCOEwns6KCJxo4ua_jW5hIvjjFI/edit?usp=sharing

Quelques idées si vous n'êtes pas inspirés :
- essayer d'autres hyperparamètres (learning rate, taille des minibatch, nombre de couches...)
- regarder ce qu'il se passe si on utilise des variables non normalisées
- ajouter d'autres variables en entrée

# Votre modèle

## Rappel des variables explicatives à disposition

In [ ]:
# Initialement
x_train_full.columns

## Choix des variables explicatives

On sélectionne les variables que l'on souhaite conserver en précisant simplement à quelle catégorie elles appartiennent.

In [ ]:
# Sélectionnons un sous-ensemble de ces variables

######### TO DO #########

# Prenez ce que vous voulez dans "x_light_columns"
x_light_columns = [elt for elt in x.columns if elt not in 
                   ["ds", "temperature_real_24h_avant", "temperature_prevue", "conso_real_24h_avant"]
                  ]
#########################

x_train_light = x_train_full[x_light_columns]
x_validation_light = x_validation_full[x_light_columns]
x_test_light = x_test_full[x_light_columns]

print(x_train_light.columns)

## Création du réseau de neurones, hyper-paramétrage

Vous pouvez jouer sur l'architecture de votre reseau de neurones ici en précisant le nombre de couches et la taille des couches dans le vecteur hiddenLayers

In [ ]:
n_inputs = x_train_light.shape[1]  # nombre d'entrées du modèle
n_outputs = y_train_light.shape[1]

######### TO DO #########
# votre choix  (nombre de couche et taille des couches)
hidden_layers = [n_inputs, n_inputs, n_inputs, n_inputs, n_inputs]

# learning rate
lr = 0.01

# nombre d'epoch (= nombre de fois ou chaque element du jeu de données sera utilisé pour "apprendre")
nb_epochs = 100

# batch size (= nombre de lignes de la base de données qui seront utilisées pour calculer 
# les gradients lors d'une iteration d'apprentissage)
batch_size = 64
#########################

# NB: le nombre total "d'iteration de descente de gradient" est donc
# (taille base apprentissage / batch_size) * nb_epochs

mon_reseau_de_neurones = new_keras_model(n_inputs, n_outputs, hidden_layers)

In [ ]:
# on affiche le nombre de paramètres de votre modèle avec la fonction "summary" de Keras
mon_reseau_de_neurones.summary()

In [ ]:
mon_reseau_de_neurones.compile(
    loss='mean_squared_error', 
    optimizer=Adam(lr=lr),  # <=  TODO : vous pouvez jouer avec ça aussi
)

# Tensorboard
Notre utilitaire de tensorflow qui permet de visualiser en temps réel les courbes d'apprentissage des réseau de neurones et est donc utile pour arrêter l'apprentissage si les progrès sont faibles.

In [ ]:
# Donner un nom a votre modele pour le retrouver dans les logs tensorboard
model_name = "my_own_model_" + datetime.datetime.now().strftime("%H-%M-%S")
tensorboard = TensorBoard(log_dir="logs/{}".format(model_name),histogram_freq=1)


#lancement dans le notebook
#vous pouvez rafraîchir tensorboard lorsque un modèle est entraîné pour voir la progression tensorboard 
%tensorboard --logdir logs 

## Entrainement

La cellule suivante peut prendre un peu de temps à s'exécuter.

In [ ]:
mon_reseau_de_neurones.fit(
    x_train_light, 
    y_train_light, 
    epochs=nb_epochs,
    batch_size=batch_size,
    validation_data=(x_validation_light, y_validation_light),
    callbacks=[tensorboard]
)

## Evaluation de la qualité du modèle

In [ ]:
predictions_train_scaled = mon_reseau_de_neurones.predict(x_train_light)
predictions_train = scaler_conso_nat.inverse_transform(predictions_train_scaled).reshape(-1)

predictions_val_scaled = mon_reseau_de_neurones.predict(x_validation_light)
predictions_val = scaler_conso_nat.inverse_transform(predictions_val_scaled).reshape(-1)

predictions_test_scaled = mon_reseau_de_neurones.predict(x_test_light)
predictions_test = scaler_conso_nat.inverse_transform(predictions_test_scaled).reshape(-1)

print(predictions_test)

In [ ]:
iplot([{"x": y_test_full['ds'], "y": y_test_full['conso_real'], "name": "realise"},
       {"x": y_test_full['ds'], "y": predictions_test, "name": "prevision"}
      ])

In [ ]:
relative_error_on_train = np.abs((y_train_full['conso_real'] - predictions_train) / y_train_full['conso_real'])
mean_error_on_train = np.mean(relative_error_on_train)
max_error_on_train = np.max(relative_error_on_train)
rmse = np.sqrt(mean_squared_error(y_train_full['conso_real'], predictions_train))

print("Erreur moyenne sur le jeu de train : " + str(mean_error_on_train * 100) + " %")
print("Erreur max sur le jeu de train : " + str(max_error_on_train * 100) + " %")
print("RMSE : " + str(rmse) + " MW")

In [ ]:
relative_error_on_val = np.abs((y_validation_full['conso_real'] - predictions_val) / y_validation_full['conso_real'])
mean_error_on_val = np.mean(relative_error_on_val)
max_error_on_val = np.max(relative_error_on_val)
rmse = np.sqrt(mean_squared_error(y_validation_full['conso_real'], predictions_val))

print("Erreur moyenne sur le jeu de validation : " + str(mean_error_on_val * 100) + " %")
print("Erreur max sur le jeu de validation : " + str(max_error_on_val * 100) + " %")
print("RMSE : " + str(rmse) + " MW")

In [ ]:
relative_error_on_test = np.abs((y_test_full['conso_real'] - predictions_test) / y_test_full['conso_real'])

mean_error_on_test = np.mean(relative_error_on_test)
max_error_on_test = np.max(relative_error_on_test)
rmse = np.sqrt(mean_squared_error(y_test_full['conso_real'], predictions_test))

print("Erreur moyenne sur le jeu de test : " + str(mean_error_on_test * 100) + " %")
print("Erreur max sur le jeu de test : " + str(max_error_on_test * 100) + " %")
print("RMSE : " + str(rmse) + " MW")

In [ ]:
iplot([{"x": y_test_full['ds'], "y": 100. * relative_error_on_test, "name": "erreur relative (%)"}])

## Pour aller encore plus loin

Le modèle ci-dessus peut être rendu encore plus performant par exemple en considérant des features comme "jour d'avant vacances", "jour d'après vacances"... 

Passer du temps à tuner les hyper-paramètres serait certainement bénéfique aussi.

De manière assez surprenante, élargir le réseau de neurones pour prédire les consommations régionales peut également améliorer la qualité de la prédiction de l'échelle nationale. C'est l'idée du multi-tasking. Pour intégrer ces données supplémentaire, il est nécessaire de passer un peu de temps pour repréparer les données : aussi rendez-vous dans le TP *preparation_donnees.ipynb*.

On pourra également considérer en sortie du modèle non pas la prédiction pour juste 24 heures plus tard, mais plutôt pour une plage horaire [1 heure plus tard, ..., 24 heures plus tard]. Ceci permet de capter des dynamiques. Un réseau de neurones de type convolutionnel serait aussi une option crédible.